In [7]:
import dagshub
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/aryan0147/Capstone-Project-2.mlflow")

dagshub.init(repo_owner='aryan0147', repo_name='Capstone-Project-2', mlflow=True)

Initialized MLflow to track repo "aryan0147/Capstone-Project-2"

Repository aryan0147/Capstone-Project-2 initialized!

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna

In [9]:
# Set or create an experiment
mlflow.set_experiment("ML-Algos")

<Experiment: artifact_location='mlflow-artifacts:/d43192cc72bb48deb03d07938721755b', creation_time=1739197271560, experiment_id='12', last_update_time=1739197271560, lifecycle_stage='active', name='ML-Algos', tags={}>

In [10]:
df = pd.read_csv('../reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [12]:
import mlflow
import optuna
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 2: TF-IDF vectorization setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

# **Step 3: Train-test split BEFORE any class balancing (Prevents data leakage)**
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# **Step 4: Fit TF-IDF only on training data**
X_train_vec = vectorizer.fit_transform(X_train)  # Fit only on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data without refitting

# **Step 5: Compute Class Weights Instead of SMOTE**
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ClassWeights_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # Logistic Regression with computed class weights
    model = LogisticRegression(
        C=C, penalty=penalty, solver='liblinear', class_weight=class_weight_dict, random_state=42
    )

    # Fit and evaluate the model
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(
        C=best_params['C'], penalty=best_params['penalty'], solver='liblinear',
        class_weight=class_weight_dict, random_state=42
    )

    # Log the best model with MLflow
    log_mlflow("LogisticRegression", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2025-02-10 20:14:53,716] A new study created in memory with name: no-name-40f20da8-da4b-450b-8fcf-219fec528c8d
[I 2025-02-10 20:14:53,794] Trial 0 finished with value: 0.6285285694804309 and parameters: {'C': 0.00525699913851439, 'penalty': 'l2'}. Best is trial 0 with value: 0.6285285694804309.
[I 2025-02-10 20:14:53,887] Trial 1 finished with value: 0.7098049911359607 and parameters: {'C': 0.025518589739789477, 'penalty': 'l2'}. Best is trial 1 with value: 0.7098049911359607.
[I 2025-02-10 20:14:53,909] Trial 2 finished with value: 0.22501022773762444 and parameters: {'C': 0.00010156887451916792, 'penalty': 'l1'}. Best is trial 1 with value: 0.7098049911359607.
[I 2025-02-10 20:14:54,089] Trial 3 finished with value: 0.789581344606573 and parameters: {'C': 0.9879956804585234, 'penalty': 'l2'}. Best is trial 3 with value: 0.789581344606573.
[I 2025-02-10 20:14:54,117] Trial 4 finished with value: 0.4301104595663439 and parameters: {'C': 0.0004949830876919085, 'penalty': 'l1'}. Best 

🏃 View run LogisticRegression_ClassWeights_TFIDF_Trigrams at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12/runs/6f5f355cdb9f4135b286b624821df136
🧪 View experiment at: https://dagshub.com/aryan0147/Capstone-Project-2.mlflow/#/experiments/12
